In [8]:
# Initialize the random number generator
rng = MersenneTwister(2016);

In [9]:
function pretty_print(v)
    @printf "["
    for i=1:length(v)
        @printf "%8.5f" v[i]
        if i < length(v)
            @printf "  "
        end
    end
    @printf "]\n"
end    

pretty_print (generic function with 1 method)

In [10]:
# Size of matrix
n = 4
Q = rand(rng, n, n)
Q, = qr(Q)

Λ = diagm(Float32[4.0^(-i) for i=0:n-1])

A = Q * Λ * Q'

4x4 Array{Float64,2}:
  0.4243     0.325641   0.290424  -0.0383016
  0.325641   0.327093   0.278628   0.0323107
  0.290424   0.278628   0.438948   0.115986 
 -0.0383016  0.0323107  0.115986   0.137785 

In [11]:
r = 2
Qk = rand(rng, n, r)

for k=1:4
    Qk = A * Qk
    Qk, Rk = qr(Qk)
    for j=1:2
        if Qk[1,j] * Q[1,j] < 0
            Qk[:,j] = -Qk[:,j]
            # This is not necessary but avoids conflicting signs later
        end
    end
    println("\nIteration ",k)
    pretty_print(Qk[:,1])
    pretty_print(Q[:,1])
    pretty_print(Qk[:,2])
    pretty_print(Q[:,2])
end


Iteration 1
[-0.51027  -0.49695  -0.67445  -0.19436]
[-0.59721  -0.53726  -0.59106  -0.07311]
[ 0.51902   0.39708  -0.52932  -0.54108]
[ 0.49054   0.13115  -0.53093  -0.67844]

Iteration 2
[-0.57508  -0.53058  -0.61413  -0.10301]
[-0.59721  -0.53726  -0.59106  -0.07311]
[ 0.50480   0.19864  -0.53580  -0.64703]
[ 0.49054   0.13115  -0.53093  -0.67844]

Iteration 3
[-0.59174  -0.53575  -0.59692  -0.08061]
[-0.59721  -0.53726  -0.59106  -0.07311]
[ 0.49455   0.14800  -0.53250  -0.67079]
[ 0.49054   0.13115  -0.53093  -0.67844]

Iteration 4
[-0.59585  -0.53689  -0.59253  -0.07499]
[-0.59721  -0.53726  -0.59106  -0.07311]
[ 0.49157   0.13536  -0.53135  -0.67654]
[ 0.49054   0.13115  -0.53093  -0.67844]


In [12]:
Qk = rand(rng, n, r)
niter = 30
err = zeros(niter,2)

for k=1:niter
    Qk = A * Qk
    Qk, Rk = qr(Qk)
    for j=1:2
        if Qk[1,j] * Q[1,j] < 0
            Qk[:,j] = -Qk[:,j]
        end
        err[k,j] = norm(Qk[:,j] - Q[:,j])
    end
end
err

30x2 Array{Float64,2}:
 0.126301     1.15456    
 0.0313297    0.602246   
 0.00782845   0.173239   
 0.00195705   0.0437703  
 0.000489262  0.0109499  
 0.000122315  0.00273759 
 3.05788e-5   0.000684399
 7.64471e-6   0.0001711  
 1.91118e-6   4.2775e-5  
 4.77794e-7   1.06937e-5 
 1.19449e-7   2.67344e-6 
 2.98622e-8   6.68359e-7 
 7.46554e-9   1.6709e-7  
 ⋮                       
 1.82266e-12  4.07936e-11
 4.55891e-13  1.01985e-11
 1.1406e-13   2.54969e-12
 2.8662e-14   6.37648e-13
 7.13232e-15  1.59609e-13
 1.76592e-15  4.01093e-14
 3.98609e-16  1.02808e-14
 2.32634e-16  2.81962e-15
 1.11886e-16  9.22221e-16
 1.11022e-16  5.55112e-16
 1.11022e-16  3.33067e-16
 1.11022e-16  3.84593e-16

In [13]:
using PlotlyJS

In [14]:
sc1 = scatter(y=err[:,1],name="q1 err")
sc2 = scatter(y=err[:,2],name="q2 err")
sc3 = scatter(y=Float32[4.0^(-i) for i=1:niter],name="Theory",line_dash="dash")
plot([sc1, sc2, sc3],Layout(yaxis_type="log",xaxis_title="Iteration",yaxis_title="Error"))